In [23]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
import re
import datetime
import h5py
import os
from astropy.time import Time
import matplotlib.pyplot as plt
import pickle


def is2sitdat4_to_gdf(target_files):
    gdf_list = list()
    m=0
    for file in target_files:
        m = m +1
        print(m/len(target_files))
        data = nc.Dataset(file)
        start_idx = os.path.basename(file).find("bnum") 
        end_idx = start_idx + 5
        beam = os.path.basename(file)[start_idx:end_idx]

        d = {
            'latitude': np.array(data["latitude"]),
            'longitude': np.array(data["longitude"]),
            'seg_length': np.array(data["seg_length"]),
            'ice_thickness_W99mod5dist': np.array(data["ice_thickness_W99mod5dist"]),
            'sea_ice_thickness': np.array(data["ice_thickness"]),
            'sea_ice_thickness_unc': np.array(data["ice_thickness_unc"]),
            'snow_depth': np.array(data["snow_depth"]),
            'ice_type': np.array(data["ice_type"]),
            'ssh_flag': np.array(data["ssh_flag"]),
            'region_flag': np.array(data["region_flag"]),
            'time': Time(np.array(data["gps_seconds"]), format='gps').to_datetime(),
            'beam': beam,
        }
        df = pd.DataFrame(data=d)
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
        gdf = gdf.to_crs('EPSG:3413')
        gdf = gdf[gdf['latitude'] > 50.0]
        gdf_list.append(gdf)

    gdf_final = pd.concat(gdf_list).pipe(gpd.GeoDataFrame)
    gdf_final.crs = gdf_list[0].crs
    return gdf_final.reset_index(drop=True)

def get_file_list(directory):
    file_list = glob.glob(directory + '/**/*.nc', recursive=True)
    return file_list

def get_file_dates(file_list):
    date_str = '{14}'
    date_pt = '%Y%m%d%H%M%S'

    dates = [
        datetime.datetime.strptime(re.search(r'\d' + date_str, file).group(), date_pt)
        for file in file_list
    ]
    return dates

def get_target_files(file_list, dates, t0, t1):
    files=[]
    for i in range(len(dates)):
        if t0 <= dates[i] < t1:
            files.append(file_list[i])
    return files

In [17]:
directory = '/Volumes/data2/projects/chione/data/icesat2/is2sitdata'
file_list = get_file_list(directory)
file_dates = get_file_dates(file_list)

In [24]:
out_dir = '/Users/rricker/Documents/projects/NFR_TARDIS/results/'

t0 = datetime.datetime(2022, 4, 1, 0, 0, 0)
t1 = t0 + datetime.timedelta(days= 30)

day=[]
seg_length = []
sit=[]

target_files = get_target_files(file_list, file_dates, t0, t1)
#data = is2sitdat4_to_gdf(target_files)

#for bm in data.beam.unique():
#    tmp = data[data['beam']==bm]
#    with open(out_dir + bm + "_apr.pkl", 'wb') as f:
#        pickle.dump(tmp, f)

In [25]:
target_files

['/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401012104_01371501_005_01_bnum1gt1l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401012104_01371501_005_01_bnum3gt2l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401012104_01371501_005_01_bnum5gt3l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401025521_01381501_005_01_bnum1gt1l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401025521_01381501_005_01_bnum3gt2l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401025521_01381501_005_01_bnum5gt3l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401042939_01391501_005_01_bnum3gt2l_001.nc',
 '/Volumes/data2/projects/chione/data/icesat2/is2sitdata/2022/04/IS2SITDAT4_01_20220401042939_013

In [10]:
import yaml
with open('/Users/rricker/GitLab/projects/chione/config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
sensor = config["options"]["sensor"]
target_var = config["options"]["target_variable"]
stk_opt = config['options']['proc_step_options']['stacking']

i=0

t0 = stk_opt['t_start'] + datetime.timedelta(days=i)
t1 = stk_opt['t_start'] + datetime.timedelta(days=i + 1)

In [42]:
%%time
hem = 'IS2SITDAT4_01'
directory = config['dir'][sensor]['level2'][target_var]
file_list = [file_path for file_path in glob.iglob(os.path.join(directory, "**", "*"), recursive=True)
             if hem.lower() in os.path.basename(file_path.lower())]

date_str = '{14}'
date_pt = '%Y%m%d%H%M%S'

dates = [
    datetime.datetime.strptime(re.search(r'\d' + date_str, file).group(), date_pt)
    for file in file_list
]
file_dates = dates


dates = file_dates
#target_files = [file_list[dates.index(d)] for d in dates if t0 <= d < t1]
target_files = [file for date, file in zip(dates, file_list) if t0 <= date < t1]



CPU times: user 96.6 ms, sys: 8.44 ms, total: 105 ms
Wall time: 104 ms
